# Paysim

From Kaggle"s site **[Synthetic Financial Datasets For Fraud Detection](https://www.kaggle.com/ntnu-testimon/paysim1)**

>PaySim simulates mobile money transactions based on a sample of real transactions extracted from one month of financial logs from a mobile money service implemented in an African country. The original logs were provided by a multinational company, who is the provider of the mobile financial service which is currently running in more than 14 countries all around the world.

>This synthetic dataset is scaled down 1/4 of the original dataset and it is created just for Kaggle.

The following notebook rewrites the original using the ```mlrun``` package. At the end of this demonstration we will have a fraud report covering any period within the last 30 days, and a deployed fraud detection service.

## Data Description

**This is a sample of 1 row with headers explanation:**

1,PAYMENT,1060.31,C429214117,1089.0,28.69,M1591654462,0.0,0.0,0,0

**step** - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

**type** - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

**amount** - amount of the transaction in local currency.

**nameOrig** - customer who started the transaction

**oldbalanceOrg** - initial balance before the transaction

**newbalanceOrig** - new balance after the transaction

**nameDest** - customer who is the recipient of the transaction

**oldbalanceDest** - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

**newbalanceDest** - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

**isFraud** - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

**isFlaggedFraud** - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

In [1]:
from os import path, makedirs, getenv
import time

import pandas as pd
import numpy as np
import v3io_frames as v3f

import pyarrow.parquet as pq
import pyarrow as pa

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

### timing a tsdb read and write

In [2]:
# %%time
# paysim = pd.read_csv(path.join(target_path, "PS_20174392719_1491204439457_log.csv.zip"))

#### Creating a time index

The raw data only contains a ```step``` variable, which represents a time period of 1 hour.  There are 742 unique steps in the data, which is approximately 1 month.  Here we will translate (map) the step values into a ```DateTime ```s:

In [3]:
import v3io_frames as v3f
client = v3f.Client("framesd:8081", container="users")

# Relative path to the TSDB table within the parent platform data container
tsdb_table = path.join(getenv("V3IO_USERNAME") + "/projects/paysim/tsdb_tbl")

In [4]:
if path.isdir(tsdb_table):
    print("found existing table, deleting")
    # takes too long... just delete folder
    client.delete("tsdb", tsdb_table)

client.create(backend="tsdb", table=tsdb_table, attrs={"rate": "1/s"})

In [5]:
# create a time sequence of the required length, there are <744 periods in the
# raw data
df = pd.DataFrame(
        {"hours": pd.date_range("2019-01-01", freq="1H", periods=744)})

# Create the mappings ```dict``` where each unique step is mapped to a start time
time_mappings = {}
for a,b in zip(range(744), df.hours):
    time_mappings[a] = b

In [ ]:
%%time

reader = pd.read_csv(
    path.join('/User/projects/paysim/data', "PS_20174392719_1491204439457_log.csv.zip"),
    chunksize=1_000)

timeproc = []
for (i, chunk) in enumerate(reader):
    start = time.time()
    chunk.step = chunk.step.map(time_mappings)
    chunk.rename(columns={"step":"time"}, inplace=True)
    chunk.set_index(["time"], inplace=True)
    client.write(backend="tsdb", table=tsdb_table, dfs=chunk)
    tm = time.time()-start
    timeproc.append(int(tm))

In [ ]:
# display ingestion rates for blocks using grafana